In [2]:
import cv2
import mediapipe as mp
import os

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Folder containing the videos
input_folder = 'vid'

# Get the list of video files in the folder
video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]

for video_file in video_files:
    # Open the video file
    cap = cv2.VideoCapture(os.path.join(input_folder, video_file))
    
    # Extract the base name and the video number
    video_name = os.path.splitext(video_file)[0]
    base_name = video_name.split('_')[0]

    # Create the output folder based on the base name if it doesn't exist
    output_folder = os.path.join('processed_frames', base_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Calculate FPS change
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip = max(1, round(original_fps / 15))

    frame_idx = 0
    yyy = 1
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit if the video ends
        
        # Process every nth frame to adjust the FPS
        if frame_idx % frame_skip == 0:
            # Convert the frame to RGB for Mediapipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process the frame for pose detection
            pose_results = pose.process(frame_rgb)

            # Check if landmarks are detected
            if pose_results.pose_landmarks:
                # Draw the skeleton on the frame
                mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Save the frame with the specified naming convention
            output_frame_path = os.path.join(output_folder, f"{video_name}_{yyy}.jpg")
            cv2.imwrite(output_frame_path, frame)
            yyy += 1
        
        frame_idx += 1

    cap.release()

print("Processing completed.")


Processing completed.
